In [44]:
import json
from pathlib import Path
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager as cdm
from bs4 import BeautifulSoup as bs

In [1]:
url = "https://solved.ac/problems/level/{level}/unsolved?page={page}"
level = int(input("level: "))
if level > 30:
    level = 30
elif level < 1:
    level = 1
def page(p):
    return url.format(level=level, page=p)

In [56]:
tiers = ["Bronze", "Silver", "Gold", "Platinum", "Diamond", "Ruby"]
tier = tiers[level // 5]
tier_lv = 6 - level % 5
print(tier, tier_lv)

Bronze 4


In [82]:
fp = Path("./")
cache_path = fp / "cache" / f"{level}"
cache_path.mkdir(parents=True, exist_ok=True)
crawl_path = fp / tier / f"{tier_lv}.ipynb"
crawl_path.parent.mkdir(parents=True, exist_ok=True)
if not crawl_path.exists():
    default_ipynb = r'{"cells":[],"metadata":{}}'
    with open(crawl_path, "w", encoding="utf-8") as f:
        f.write(default_ipynb)

In [83]:
driver = webdriver.Chrome(cdm().install())
driver.get('https://solved.ac/login')
print(driver.title)
input("브라우저에서 솔브닥 로그인 후 엔터를 눌러주세요.")

/var/folders/5v/wxprbmzx3yv6xbk39x3shz0m0000gn/T/ipykernel_17926/787701786.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdm().install())


solved.ac


''

In [19]:
def save_source(path, source):
    with open(path, "w", encoding="utf-8") as f:
        f.write(source)

In [71]:
def get_path(p=0):
    driver.get(page(p))
    source = driver.page_source
    if source.find("해당하는 문제가 없습니다.") - source.find("시프트 마음대로") < 100:
        return
    path = cache_path / f"{p}.html"
    save_source(path, source)
    yield path
    yield from get_path(p + 1)

In [72]:
paths = list(get_path())
print(f"{len(paths)} 페이지 저장됨")

Crawl 6 pages


In [75]:
def create_md_cell(header):
    return {
        "cell_type": "markdown",
        "metadata": {},  # metadata 까지 있어야 오류가 안 남
        "source": [header],
    }

In [76]:
def create_empty_code_cell():
    return {
        "cell_type": "code",
        "metadata": {},  # metadata 까지 있어야 오류가 안 남
        "source": [""],
    }

In [77]:
def crawl_from_cache(cache_path):
    with open(cache_path, "r") as f:
        raw_html = f.read()
    soup = bs(raw_html, "html.parser")
    title_link = [
        (
            elem.select_one("span").text,
            elem.select_one("a")["href"],
        )
        for elem in soup.select(".css-d6mf5j")
    ]
    headers = [f"## [{title}]({link})" for title, link in title_link]
    new_cells = sum(
        (
            [
                create_md_cell(header),
                create_empty_code_cell(),
            ] for header in headers
        ),
        start=[],
    )
    return new_cells

In [78]:
with open(crawl_path, "r") as f:
    raw = f.read()
data = json.loads(raw)
cells = data["cells"]

In [79]:
new_cells = sum((crawl_from_cache(path) for path in paths), start=[])
data["cells"] = new_cells + cells

In [80]:
with open(crawl_path, "w") as f:
    json.dump(data, f)

In [81]:
if input("브라우저를 종료하시겠습니까? (ㅇ 시 종료): ") == "ㅇ":
    driver.close()